<a href="https://colab.research.google.com/github/JoyAssis/womakerscode_dataAnalitics_teste-de-hopoteses/blob/main/teste_hipoteses_analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste de Hipóteses - Análise Estatística com Python

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp, ttest_ind, f_oneway, chi2_contingency, pearsonr, t, shapiro, norm

# Carregando os dados
experimento_df = pd.read_csv('/content/experimento_teste_ab.csv')
pacientes_df = pd.read_csv('/content/pacientes.csv')

## Exercício 1: Maior taxa de conversão

In [2]:
agrupado = experimento_df.groupby('Versão_Página').agg(
    total_visualizacoes=('Visualizações', 'sum'),
    total_conversoes=('Conversões', 'sum')
)
agrupado['taxa_conversao_%'] = (agrupado['total_conversoes'] / agrupado['total_visualizacoes']) * 100
agrupado

,total_visualizacoes,total_conversoes,taxa_conversao_%
Versão_Página,,,
A,150835,105,0.069612
B,150676,125,0.082959


## Exercício 2: Cálculo do tamanho da amostra

In [3]:
p1 = agrupado.loc['A', 'taxa_conversao_%'] / 100
p2 = 0.10
p_bar = (p1 + p2) / 2
z_alpha = norm.ppf(1 - 0.05 / 2)
z_beta = norm.ppf(0.80)
n = (2 * ((z_alpha + z_beta)**2) * p_bar * (1 - p_bar)) / (p2 - p1)**2
round(n)

76

## Exercício 3: Teste t para média de idades

In [4]:
indices_amostra = [909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381,
793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967, 699, 473, 222, 89, 639, 883,
558, 757, 84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362]
amostra_idades = pacientes_df.loc[indices_amostra, 'Idade']
t_stat, p_bi = ttest_1samp(amostra_idades, popmean=50)
p_uni = p_bi / 2
amostra_idades.mean(), t_stat, p_uni

(np.float64(50.08888888888889),
 np.float64(0.052280983519350496),
 np.float64(0.4792707233816392))

## Exercício 4: Tipo de Amostra

A amostra é **independente**, pois compara dois grupos distintos sem pareamento.

## Exercício 5: Teste t entre dois grupos

In [5]:
indices_5 = [690, 894, 67, 201, 364, 19, 60, 319, 588, 643, 855, 623, 530, 174,
105, 693, 6, 462, 973, 607, 811, 346, 354, 966, 943, 372]
amostra_5 = pacientes_df.loc[indices_5]
grupo_cond = amostra_5[amostra_5['Estado_Saude'] == 1]['Pressao_Arterial']
grupo_saud = amostra_5[amostra_5['Estado_Saude'] == 0]['Pressao_Arterial']
t_stat, p_valor = ttest_ind(grupo_cond, grupo_saud, equal_var=False)
grupo_cond.mean(), grupo_saud.mean(), t_stat, p_valor

(np.float64(153.19725449805927),
 np.float64(152.94305165531088),
 np.float64(0.08304391716460979),
 np.float64(0.9345291578970931))

## Exercício 6: Teste ANOVA

In [6]:
grupos_etnia = pacientes_df['Nome_Etnia'].unique()
dados_etnia = [pacientes_df[pacientes_df['Nome_Etnia'] == e]['Pressao_Arterial'] for e in grupos_etnia]
f_stat, p_valor = f_oneway(*dados_etnia)
f_stat, p_valor

(np.float64(0.23245296261596837), np.float64(0.873800123850745))

## Exercício 7: Teste Qui-Quadrado

In [7]:
contingencia = pd.crosstab(pacientes_df['Nome_Genero'], pacientes_df['Nome_Estado_Saude'])
chi2, p, dof, expected = chi2_contingency(contingencia)
contingencia, chi2, p

(Nome_Estado_Saude  Com condições de saúde adicionais  Saudável
 Nome_Genero                                                   
 Feminino                                         257       246
 Masculino                                        253       244,
 np.float64(0.0),
 np.float64(1.0))

## Exercício 8: Correlação entre idade e pressão arterial

In [8]:
correlacao, p_valor = pearsonr(pacientes_df['Idade'], pacientes_df['Pressao_Arterial'])
correlacao, p_valor

(np.float64(-0.02138535693464897), np.float64(0.49936082120556174))

## Exercício 9: Intervalo de confiança para pressão arterial

In [9]:
grupo = pacientes_df[pacientes_df['Estado_Saude'] == 1]['Pressao_Arterial']
media = grupo.mean()
std = grupo.std()
n = grupo.count()
t_crit = t.ppf(1 - 0.05/2, df=n-1)
margem = t_crit * (std / np.sqrt(n))
(media - margem, media + margem)

(np.float64(149.21972443653837), np.float64(151.01717945377672))

## Exercício 10: Teste de normalidade (Shapiro-Wilk)

In [10]:
shapiro(pacientes_df['Pressao_Arterial'])

ShapiroResult(statistic=np.float64(0.9970366758795889), pvalue=np.float64(0.060729578232028174))